In [38]:
import numpy as np                        # базовые модули
import matplotlib.pyplot as plt
import pandas as pd 
import scipy.stats as stats

import nltk                               # для обработки текстов 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

from datetime import datetime 

import warnings
warnings.filterwarnings("ignore")

В этой тетрадке мы первично обрабытываем исходный датасет с постами, собранный парсером

In [39]:
data = pd.read_csv('data.csv').drop('Unnamed: 0', axis = 1)

In [27]:
no_text_idx = data[data.text.isna()].index

In [28]:
with_text = data.drop(index = no_text_idx)

In [30]:
with_text['text'] = with_text['text'].apply(lambda x: ' '.join(x.split('\n')))

In [31]:
with_text.reset_index(inplace = True)

In [32]:
with_text.drop('index', axis = 1, inplace = True)

Лемматизация -- приведем слова во всех постах в начальную форму.

In [33]:
from pymystem3 import Mystem

mystem_analyzer = Mystem()

Не трогайте ячейку ниже, она отрабатывалась полтора часа)

In [43]:
with_text['text'] = with_text['text'].apply(lambda x: ' '.join(mystem_analyzer.lemmatize(x)[:-1]))

In [45]:
with_text.to_csv('lemmatized.csv')

In [43]:
with_text = pd.read_csv('lemmatized.csv')

In [44]:
with_text.text[214]

'подруга  -  это   святой !  особенно ,  если   она   звать   леся'

In [45]:
nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)

True

Избавимся от пунктуации и стоп-слов в тексте.

In [46]:
noise = set(stopwords.words('russian') + stopwords.words('english') + list(punctuation))

In [47]:
def stopwords_dropper(string):
    noise = set(stopwords.words('russian') + stopwords.words('english') + list(punctuation + '—–'))  
    
    text = str(string).split() # лемматизатор уже отделил всю пунктуацию пробелами
    out = []
    
    for word in text:
        if word in noise:
            continue
        else:
            out.append(word)
            
    return ' '.join(out)

In [48]:
wo_stopwords = with_text['text'].apply(stopwords_dropper)

In [49]:
data_no_stopwords = with_text[['author', 'date', 'n_likes', 'n_reposts', 'with_photo', 'is_repost?']]

In [50]:
data_no_stopwords['text'] = wo_stopwords

Дополнительно сохраним файл без шумов. Но сначала переведем даты в формат datetime

In [51]:
def datetimer(string):
    month_dict = {
    'янв': '01',
    'фев': '02',
    'мар': '03',
    'апр': '04',
    'мая': '05',
    'июн': '06',
    'июл': '07',
    'авг': '08',
    'сен': '09',
    'окт': '10',
    'ноя': '11',
    'дек': '12'
    }
    date = string.split()
    day = int(date[0])
    year = int(date[2])
    month = int(month_dict[date[1]])
    date = datetime(year, month, day)
    return date

In [52]:
data_no_stopwords.date = data_no_stopwords.date.apply(datetimer)

In [53]:
data_no_stopwords.to_csv('lemmatized_no_noise.csv')